# GADM process layers
Our clients want to use their country context, so we have to edit our GADM layer and make these new country context fit into GADM layer
Here is the process:
- Locate the polygon we want to substitute in the GADM layer
- Get the surrounding countries as a new dataframe
- Go country by country and calculate the difference between them and the new country
- Update the geometries of the surrounding countries dataframe
- Replace the countries on the gadm dataframe
- Delete the old country of the gamd dataframe
- Export it into a new vector file

- import layers / variables:
    * GADM layer
    * Client country
    * Client country Acronym

- Main idea: https://gis.stackexchange.com/questions/245064/snapping-multiple-nodes-of-a-polygon-to-the-nodes-of-a-line
- Create a linestring from the polygon: https://gis.stackexchange.com/questions/290756/transforming-a-polygon-to-a-linestring
- Snap Geopandas: https://gis.stackexchange.com/questions/290092/how-to-do-snapping-in-geopandas
- https://gis.stackexchange.com/questions/203058/why-is-shapelys-snapping-geo-snaps-not-working-as-expected
- Replace rows from df to df: https://moonbooks.org/Articles/How-to-replace-rows-of-a-dataframe-using-rows-of-another-dataframe-based-on-indexes-with-pandas-/

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point, box
import os
from sys import path
import matplotlib.pyplot as plt


In [14]:
"""Create dataframes of the import data"""
os.chdir(r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\09_norway_case")

client_country_name = "NOR"

gadm_path = r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\09_norway_case\level3_gadm_404.shp"
client_path = r"C:/Users/ruben.crespo/Documents/03_tests/administrative_units/09_norway_case/level0_norway_country.shp"
gadm_gdf = gpd.read_file(gadm_path)
client_gdf = gpd.read_file(client_path)

In [4]:
# https://stackoverflow.com/questions/60780959/how-to-filter-a-geodataframe-by-geometry-type
"""Here we check the geometry type and the number of polygons 6f the import file"""
geometry_dic = {}
geometry_type = client_gdf.geom_type.unique().tolist() #we create a list of unique geometries
if len(geometry_type) > 0: #if there are more than one type we list them
    for i in geometry_type:
        geometry_gdf = client_gdf[client_gdf.geom_type == i] #new df with the geometry
        geometry_rows = geometry_gdf.shape[0] #number of elements
        geometry_dic.update({i:geometry_rows}) #add to dic
        
        
print(geometry_dic)

# geometry_str = ''.join(geometry_gdf.geom_type.unique().tolist()) #transform the list to string


{'MultiPolygon': 1}


In [4]:
"""if we have a multipolygon (lots of polygons), we dissolve it"""
client_area_gdf = client_gdf.dissolve()
print(str(client_area_gdf.geom_type)) # check again its geometry
# client_area_gdf.plot()
# client_dissolve.head()

0    Polygon
dtype: object


In [15]:
"""Extract from GADM the country regions and replace it with the country if exists"""
"""We extract from GADM the client country and dissolve it"""
gadm_client_gdf = gadm_gdf.loc[(gadm_gdf["ID_0"] == client_country_name)] #for gadm is ID_0 for our gadm is CID
gadm_client_gdf = gadm_client_gdf.dissolve()

"""Delete the client regions of gadm"""
gadm_gdf = gadm_gdf.drop(gadm_gdf[gadm_gdf.ID_0 == client_country_name].index)

"""Import the country and restart the index"""
gadm_gdf = pd.concat([gadm_gdf, gadm_client_gdf]).reset_index(drop=True)


c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)


In [ ]:
"""Here we create the GADM boundaries and the client country"""

"""We extract from GADM the client country"""
gadm_client_gdf = gadm_gdf.loc[(gadm_gdf["ID_0"] == client_country_name)] 

"""Extract from GADM the surrounding countries of the one we want to edit"""
for row_index, row in gadm_gdf.iterrows():
    if row[0] == client_country_name: #the country we are importing
        neightbour_countries_gdf = gadm_gdf[gadm_gdf.geometry.touches(row['geometry'])] #esta bien
    else:
        pass

"""Union the gadm and client layers and dissolve it"""
gadm_countries_gdf = neightbour_countries_gdf.append(gadm_client_gdf)
gadm_boundaries_gdf = gadm_countries_gdf.dissolve() #esta bien


In [6]:
"""If the country/region has a coast, do this"""
"""Diference of the client and the gadm countries"""
client_country_gs = client_area_gdf.difference(gadm_boundaries_gdf, align= False) #this is the template
client_country_gs = client_area_gdf.difference(client_country_gs, align= False)
client_country_gdf = gpd.GeoDataFrame(geometry=client_country_gs) #esta bien

In [7]:
"""Get the country list, and start doing a difference with both main and surrounding countries""" 
countries = []
for row_index, country in neightbour_countries_gdf.iterrows():
    countries.append(country[0])
countries

['FIN', 'SWE', 'RUS']

In [8]:
"""We create an empty GeoSeries to append inside the countries that are finished"""
finished_countries_gs = gpd.GeoSeries() #.reindex_like(client_area_gdf.geometry)
finished_countries_gs.crs = "EPSG:4326"

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)


In [11]:

"""Get the country"""
for i in countries:
    country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == i)]
    
    """Difference of the country and the client"""
    new_country_gs = country_gdf.difference(client_country_gdf, align= False)
    
    """Union the client country with the possible fixed countries"""
    """BEWARE: Union works with index, to make it work, it has to be one element per index value, GeoSeries with more than one geometry does not fit"""
    if len(finished_countries_gs.index) == 0:
        pass
    else:
        finished_countries_gdf = gpd.GeoDataFrame(geometry=finished_countries_gs)
        
        finished_countries_gdf = finished_countries_gdf.dissolve()
        client_country_gs = client_country_gdf.geometry.union(finished_countries_gdf, align=False)

    """Union both countries  (+ possible fixed countries)"""
    country_union_gs = new_country_gs.union(client_country_gs, align=False)
    
    """We generate the bbox to create a megapolygon with the bbox of the union"""
    offset = 1 #be careful with the projection. If meters = 100, if degrees = 1
    tolerance = 1
    bbox_df = country_union_gs.bounds + [-offset, -offset, offset, offset]
    bbox_df['minx'] = bbox_df['minx'].sub(tolerance) # opearte in single columnn
    bbox_df['miny'] = bbox_df['miny'].sub(tolerance)
    bbox_df['maxx'] = bbox_df['maxx'].add(tolerance)
    bbox_df['maxy'] = bbox_df['maxy'].add(tolerance)

    b = [box(l, b, r, t) for l, b, r, t in zip(bbox_df.minx, bbox_df.miny, bbox_df.maxx, bbox_df.maxy)] #this is the geometry
    bbox_gdf = gpd.GeoDataFrame(bbox_df, geometry=b)
    bbox_gdf.crs = "EPSG:4326"
    
    """Do a difference with both"""
    bbox_gdf = bbox_gdf.difference(country_union_gs, align= False)
    
    """Pass to gdf and explode the multiplygon"""
    bbox_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(bbox_gdf)) 
    bbox_gdf = bbox_gdf.explode(index_parts=False)

    """Calculate the area and sort by size, then, delete the first row as it is the biggest one"""
    bbox_gdf['area'] = bbox_gdf.area
    bbox_gdf.sort_values(by=['area'], ascending=False) #sort the values to size
    bbox_gdf = bbox_gdf.iloc[1: , :] #extract the first polygon, which is the biggest one
    
    """Dissolve the small polygons and union with the country of sweeden"""
    bbox_gdf = bbox_gdf.dissolve()
    new_country_gs = new_country_gs.union(bbox_gdf['geometry'], align=False)
    
    """Append new country to the finished gs list"""
    finished_countries_gs = finished_countries_gs.append(new_country_gs)

C:\Users\ruben.crespo\AppData\Local\Temp\ipykernel_10272\380267381.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox_gdf['area'] = bbox_gdf.area
c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:551: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val = getattr(super(GeoSeries, self), mtd)(*args, **kwargs)


In [9]:
"""For testing"""
country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == "FIN")] #"FIN", SWE RUS

"""Difference of the country and the client"""
new_country_gs = country_gdf.difference(client_country_gdf, align= False)


In [11]:
"""Update the neighbours gdf with the gs"""
neightbour_countries_gdf = neightbour_countries_gdf.set_geometry(finished_countries_gs)

In [11]:
"""Replace the new countries with the old ones in the gadm dataframe"""
new_gadm_gdf = gadm_gdf.loc[neightbour_countries_gdf.index, :] = neightbour_countries_gdf[:]

In [12]:
"""Do the exportation of the new geodataframe"""
neightbour_countries_gdf.to_file('neightbour_countries_region.shp', driver="ESRI Shapefile")

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
